In [1]:
# TEST SET (reloading independent code fragment for now sampled excel)

# Import CNN Zero-Shot & other necessary stuff
from transformers import pipeline  # It takes time here

# For data input and data cleaning
import pandas as pd
from nltk.tokenize import regexp_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from numpy import NaN
import time

# Available directories (input yours for personal use)
cep_data_dir = "/home/aan0709@tmme/pcu-research-mapping/data/"

working_dir = "TEST/"  # CHANGE WORKING DIRECTORY INSIDE IEEE XPLORE FOLDER


# For data importing
def openExcel(excel_name, sheet_name, directory):
    df = pd.read_excel(directory + excel_name + ".xlsx", sheet_name=sheet_name)
    return df


def cleanIndexes(df):
    temp_ls = df.columns
    for i in df.columns:
        if i == "abstract":
            break
        df.drop(columns=[i], inplace=True)
    return df


# Select INPUT EXCEL
file_name = "TESTSET_Charger_answered"  # -> I have edited the Affiliations and the Keywords part

df = openExcel(file_name, "Sheet1", cep_data_dir + working_dir)

df = cleanIndexes(df)
df.reset_index(drop=True, inplace=True)

In [2]:
start = time.time()

classifierGPU0 = pipeline(
    "zero-shot-classification", model="facebook/bart-large-mnli", device=1
)
# classifierGPU0 = pipeline("zero-shot-classification", model='roberta-large-mnli', device=-1)
# classifierGPU = pipeline("zero-shot-classification", model='xlm-roberta-large-finetuned-conll03-english', device=-1)
# classifierGPU = pipeline("zero-shot-classification", model='finiteautomata/beto-sentiment-analysis', device=-1)
# classifierGPU = pipeline("zero-shot-classification", model='cross-encoder/nli-distilroberta-base', device=-1)

end = time.time()
print("seconds elapsed: ", end - start)

seconds elapsed:  11.05048656463623


In [19]:
# TEST 0

positive_outcome = "onboard charger on-board charger"
# negative_outcome = "not traction inverter"

candidate_labels = [positive_outcome]  # , negative_outcome]
hypothesis_template = "The research is about {}?"  # Categorization question

In [11]:
# TEST 1

positive_outcome = "onboard charger on-board charger motor windings vehicle"
negative_outcome = "control grid"

candidate_labels = [positive_outcome, negative_outcome]
hypothesis_template = "The research is about {}?"  # Categorization question

In [16]:
# TEST 2

positive_outcome = "onboard charger on-board charger integrated charger  motor  vehicle double active bridge DAB resonant converter LLC"

candidate_labels = [
    "onboard charger",
    "on-board charger",
    "integrated charger",
    "motor",
    "vehicle",
    "double active bridge DAB",
    "resonant converter LLC",
]  # "control", "grid", "inverter", "wireless"]
hypothesis_template = "The research is about {}?"  # Categorization question

In [22]:
from stop_words import get_stop_words
from nltk.corpus import stopwords

percentages = []
for perc_loop in range(0, 100, 5):

    en_stopwords = []
    en_stopwords = list(get_stop_words("en"))  # About 900 stopwords
    nltk_words = list(stopwords.words("english"))  # About 150 stopwords
    en_stopwords.extend(nltk_words)

    lemma = WordNetLemmatizer()

    Author_Keywords = "index_terms.author_terms.terms"
    IEEE_Keywords = "index_terms.ieee_terms.terms"

    zero_shot_judgement = []
    # df["ZeroShot Judgement"] = NaN

    count_all = len(df)
    count_diff = 0

    start = time.time()

    final_judgement_dict = {}
    for i in range(len(df)):
        # for i in range(3):
        data = ""

        # Where to search
        if isinstance(df.loc[i, "abstract"], str):
            data += df.loc[i, "abstract"]
        if isinstance(df.loc[i, "title"], str):
            data += " " + df.loc[i, "title"]
        exception_to_str = df.loc[i, IEEE_Keywords]
        if isinstance(exception_to_str, str):
            exception_to_str = " ".join(df.loc[i, IEEE_Keywords].split(";"))
            data += " " + exception_to_str
        exception_to_str = df.loc[i, Author_Keywords]
        if isinstance(exception_to_str, str):
            exception_to_str = " ".join(df.loc[i, Author_Keywords].split(";"))
            data += " " + exception_to_str
            printing_data = data

        # Data Cleaning with NLTK, NumPy
        tokens = regexp_tokenize(data, pattern=r"\s|[\.,;'()]", gaps=True)
        words = []
        for k in tokens:
            if k not in en_stopwords:
                k = lemma.lemmatize(k)
            words.append(k)
        data = " ".join(words)

        # Here the data has been cleaned
        input_text = data

        # Rienforcing the data with keyword
        tokens = regexp_tokenize(data, pattern=r"\s|[\&<>\.,;'()]", gaps=True)
        for j in tokens:  # For every word in the data:
            lc_j = j.lower()
            if (
                len(j) > 3
            ):  # this excludes ie. the BJTs keyword, so be careful (but also filters out words like "and" etc)
                for k in candidate_labels:
                    lc_k = k.lower()
                    if lc_j in lc_k:
                        data += " " + j  # + " " + j)

        start_judge = time.time()
        # Extract the probabilities from the CNN
        final_judgement_dict = classifierGPU0(
            input_text,
            candidate_labels,
            hypothesis_template=hypothesis_template,
            multi_label=False,
        )

        end_judge = time.time()

        if (
            final_judgement_dict["labels"][0] in positive_outcome
            and final_judgement_dict["scores"][0] > perc_loop / 100.0
        ):
            zero_shot_judge = 1
        else:
            zero_shot_judge = 0
        # print(zero_shot_judge)
        zero_shot_judgement.append(zero_shot_judge)
        df.loc[i, "ZeroShot Judgement"] = float(zero_shot_judge)

        # print(data)

    end = time.time()

    # Print differences
    for i in range(len(df)):
        if df.loc[i, "ZeroShot Judgement"] != df.loc[i, "Manual Judgement"]:
            count_diff += 1

            """print(df.loc[i, "Manual Judgement"])
            print("----- DECISION ----- :")
            if str(zero_shot_judgement[i]) == "1":
                print("-- It is included --")
            else:
                print("-- It is NNNOOOTTT included --")    # Visual exageration to easily spot answer
            print()
            if isinstance(df.loc[i, Author_Keywords], str): auth_keys = df.loc[i, Author_Keywords].split(";")
            else: auth_keys = ""
            if isinstance(df.loc[i, IEEE_Keywords], str): ieee_keys = df.loc[i, IEEE_Keywords].split(";")
            else: ieee_keys = ""
            print("Abstract :")
            print(df.loc[i, "abstract"], "\n")
            print("Document Title :")
            print(df.loc[i, "title"], "\n")
            print("Author Keywords :")
            [print("-> ", x) for x in auth_keys]
            print()
            print("IEEE Keywords :")
            [print("-> ", x) for x in ieee_keys]"""

    count = 0
    for i in range(len(df)):
        if df.loc[i, "ZeroShot Judgement"] == "1":
            count += 1

    percentages.append((count_all - count_diff) / count_all)
    print(perc_loop / 100)
    m, s = divmod(end - start, 60)
    print("Time: %d min %0.3f sec" % (m, s))
    print(
        "From %d papers, the %d are CORRECT and the %d are WRONG ( %.2f percent correct )\n"
        % (
            count_all,
            count_all - count_diff,
            count_diff,
            (100 * (count_all - count_diff) / count_all),
        )
    )

/home/aan0709@tmme/.local/lib/python3.7/site-packages/transformers/pipelines/base.py:978: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


0.0
Time: 0 min 1.657 sec
From 50 papers, the 34 are CORRECT and the 16 are WRONG ( 68.00 percent correct )

0.05
Time: 0 min 1.626 sec
From 50 papers, the 41 are CORRECT and the 9 are WRONG ( 82.00 percent correct )

0.1
Time: 0 min 1.638 sec
From 50 papers, the 43 are CORRECT and the 7 are WRONG ( 86.00 percent correct )

0.15
Time: 0 min 1.636 sec
From 50 papers, the 46 are CORRECT and the 4 are WRONG ( 92.00 percent correct )

0.2
Time: 0 min 1.655 sec
From 50 papers, the 46 are CORRECT and the 4 are WRONG ( 92.00 percent correct )

0.25
Time: 0 min 1.656 sec
From 50 papers, the 46 are CORRECT and the 4 are WRONG ( 92.00 percent correct )

0.3
Time: 0 min 1.628 sec
From 50 papers, the 46 are CORRECT and the 4 are WRONG ( 92.00 percent correct )

0.35
Time: 0 min 1.669 sec
From 50 papers, the 46 are CORRECT and the 4 are WRONG ( 92.00 percent correct )

0.4
Time: 0 min 1.675 sec
From 50 papers, the 46 are CORRECT and the 4 are WRONG ( 92.00 percent correct )

0.45
Time: 0 min 1.653 

In [ ]:
df.loc[i, "ZeroShot Judgement"]